#### 0501_Lab  
1. Cleanse categorical data in the Student table in BTech22_\<your name\>.sqlite.   

2. Create look-up tables (aka master tables) for states, towns, and languages.    
Insert data.     
See [RTO Codes](https://www.v3cars.com/car-guide/state-code-regional-transport-office-rto-india) and 
Census Codes for [States and Cities](https://censusindia.gov.in/nada/index.php/catalog/42648) and 
[Language Codes](https://censusindia.gov.in/nada/index.php/catalog/42561)  
Use standard codes or abbreviations as PK.  



3. Create tables for electives and clubs.  
    Their PK may be auto-incremented.  
    Insert data.  